# This notebook requires data for checking the algorithm. You can upload the data from the lab 2 (about iterations) by the following code

### It will not work until you run the 6th cell, which creates the function read_xlsx.

### You can assign i to any number in the range (1,17). 

### In addition, check that lab2.xlsx is in the working directory.

###### i = int(np.random.randint(low = 1, high = 17, size = 1))

###### augm_matrix = read_xlsx(i)

###### lab(augm_matrix)

### The function lab() will work after running all cells with functions. 

# Getting working directory 

In [1]:
from os import getcwd
import numpy as np
import pandas as pd
getcwd()

'C:\\Users\\Smartguy\\Desktop\\uData.School\\Math_Tatiana'

# Procedure - Initialization of the system of equations

In [3]:
def read_xlsx(task=8):
    
    lab2 = pd.read_excel("lab2.xlsx", sheetname=str(task))
    A = lab2.iloc[:,:-1]
    A.columns=["0","1","2"]
    A = A.as_matrix()
    b = pd.DataFrame(lab2.iloc[:,-1])
    b.columns=["0"]
    b = b.as_matrix()
    augm_A = {"A":A,
              "b":b,
              "n":A.shape[0],
              "m":A.shape[1]}
    
    return augm_A

In [3]:
def dimension():

    '''
    n - number of equations, 
    m - number of variables. 
    '''
    print("\nAttention! This function processes only a square matrix of the coefficients of the system of equations \n\
with the column matrix of the constant terms.")
    
    n, m = [],[]
    n = int(input("\nPlease, enter the number of equations.\n\
number_of_equations = "))
    m = int(input("\nPlease, enter the number of variables.\n\
number_of_variables = "))
        
    return n, m

In [4]:
def initializing_system():
    
    '''
    Getting dimension of the matrix A.
    '''
    n, m = dimension()
    
    '''
    Initializing the matrix A.
    '''
    print("\nInitialized matrix A")
    A = np.zeros((n, m))
    print(A)
    
    print("\nPlease, enter elements of the matrix A by columns")
    for i in range(n):
        for j in range(m):
             A[i,j] = float(input("a" + str(i+1) + str(j+1) + '= '))
    print("\nThe matrix A\n" + str(A))            
        
    '''
    Initializing the matrix b.
    '''
    print("\nInitialized matrix b")
    b = np.zeros((n,1))
    print(b)
    
    print("\nPlease, enter elements of the matrix b")
    for i in range(n):
        b[i,0] = float(input("b"+str(i+1)+"= "))
    print("\nThe column matrix b\n"+str(b))
    
    augm_A = {"A": A,
              "b": b,
              "n": n,
              "m": m}
    
    return augm_A

# Procedure - Printing input matrix

In [5]:
def system_print(augm_matrix, settings = 0):
    
    A = pd.DataFrame(np.round_(augm_matrix["A"],12))

    if settings == 0:
        b = pd.DataFrame(np.round_(augm_matrix["b"],12))

        print("The matrix A\n" + str(A))
        print("The column matrix b\n" + str(b))
        
        if "X" in augm_matrix.keys():
            x = pd.DataFrame(np.round_(augm_matrix["X"],12))
            print("The column matrix of unknown variables x\n" + str(x))
        
    if settings == 1:
        print("The matrix A\n" + str(A))       

# Procedure - Calculating error = Ax-b

In [6]:
def error(augm_matrix, X):
    '''
    r = np.dot(A,x)-b
    '''
    A = augm_matrix["A"]
    x = X["x"]
    b = augm_matrix["b"]
    
    r = np.zeros((x.shape[0], x.shape[1]))
    r = np.dot(A,x) - b
    r = pd.DataFrame(r)

    return print("\nErrors of the solutions of the system.\n"+str(r))

# _Gauss_ procedure

### Checking whether there is a zero pivot. If it there is, sort rows and columns. If all pivots are zeros, there is no solutions 

In [7]:
def sort_by_row(A, b, i, ind_in_col):
    
    m = A.shape[1]
    interm_A = np.zeros([1,m])
    interm_b = []
    
    interm_A = A[i,:].copy()
    A[i,:] = A[ind_in_col,:]
    A[ind_in_col,:] = interm_A

    interm_b = b[i,:].copy()
    b[i,:] = b[ind_in_col,:]
    b[ind_in_col,:] = interm_b
    
    return A, b

In [8]:
def sort_x(x, i, ind_in_col):
    
    interm_x = []
    
    interm_x = x[i,:].copy()
    x[i,:] = x[ind_in_col,:]
    x[ind_in_col,:] = interm_x
    
    return x

In [9]:
def sort_by_col(A, i, ind_in_row):
    
    m = A.shape[1]
    interm_A = np.zeros([m,1])
    
    interm_A = A[:,i].copy()
    A[:,i] = A[:,ind_in_row]
    A[:,ind_in_row] = interm_A
    ind = i, ind_in_row

    return A, ind

In [10]:
def check_pivot(A, b, i, sign, error):
    '''
    error = -1 if the system is singular
    error = 0 if the system is singular and has infinitely many solutions
    error = 1 if the system is nonsingular
    '''
    ind = []
  
    if np.round_(A[i,i],12) == 0:
            ind_in_col = np.round_(np.abs(A[i:,i]),12).argmax() + i
            ind_in_row = np.round_(np.abs(A[i,i:]),12).argmax() + i
            if ind_in_col != i:
                sign *= -1
                sort_by_row(A,b,i,ind_in_col)
                print("Pivot a" + str(i+1) + str(i+1) + " is not zero")
            elif ind_in_row != i:
                sign *= -1
                A, ind = sort_by_col(A,i,ind_in_row)
                error = 0
                print("Pivot a" + str(i+1) + str(i+1) + " is not zero")                
            else:
                error = -1
                print("Pivot a" + str(i+1) +str(i+1) + " is zero")


 
    else:
        print("Pivot a" + str(i+1) + str(i+1) + " is not zero")
    
    return A, b, error, sign, ind

# Forward Elimination

In [11]:
def forward_elim(A,b,n):
    
    sign = 1
    error = 1
    ind = []

    for i in range(n-1):
        A, b, error, sign, ind = check_pivot(A,b,i,sign, error)
        if error == -1:
            print("The matrix A is singular")
            break
        else:
            for k in range(i+1,n):
                t = -A[k,i]/A[i,i]
                A[k,i] += t*A[i,i]
                A[k,i+1:] += t*A[i,i+1:]
                b[k,] += t*b[i,]
        print(np.round_(A,12))
    if error == 1:
        print("\nThe matrix A is nonsingular")
    
    augm_U = {"A":A,
              "b":b,
              "error":error,
              "sign":sign,
              "ind":ind}
   
    return augm_U

# Backward move in order to find X 

In [12]:
def backward_move(augm_U, n, m):

    A = augm_U["A"]
    b = augm_U["b"]
    error = augm_U["error"]
    sign = augm_U["sign"]
    ind = augm_U["ind"]

    x = np.zeros((m,1))
    
    if np.round_(A[n-1,m-1],12) == 0 and np.round_(b[n-1,],12) == 0:
        if len(ind) == 0:
            x[m-1,:] = float(input("There are infinitely many solutions. The last pivot is zero.\n\
Please, enter random number on your choice.\nx"+str(m-1)+"="))
        else:
            x[m-1,:] = float(input("The system is singular and has infinitely many solutions. The last pivot is zero.\n\
Please, enter random number on your choice.\nx"+str(ind[0]+1)+"="))

    elif np.round_(A[n-1,m-1],12) == 0 and np.round_(b[n-1,],12) != 0:
        print("There are no solutions. The system is singular and has no solutions")
    else:
        x[m-1,:] = b[n-1,:]/A[n-1,m-1]
    
    for i in reversed(range(n-1)):
        x[i,:] = (b[i,:]-np.dot(A[i,i:],x[i:,:]))/A[i,i]
        
    if len(ind) > 0:
        x = sort_x(x=x, i=ind[0], ind_in_col=ind[1])
  
    X = {"x":x,
         "A":A,
         "b":b,
         "error":error,
         "sign":sign}
    
    return X

# Determinant of the matrix A

In [13]:
def determinant(augm_matrix):
    
    A = augm_matrix["A"]
    sign = augm_matrix["sign"]
    n = A.shape[0]
    
    detA = 1
    for i in range(n):
        detA *= A[i,i]
    detA *= sign
    
    return detA

# Gaussian Elimination

In [14]:
def Gauss(augm_matrix):
    
    '''
    Loading input system
    '''
    A = augm_matrix["A"].copy()
    b = augm_matrix["b"].copy()
    n = augm_matrix["n"]
    m = augm_matrix["m"] 
    
    '''
    Gaussian elimination
    '''
    augm_U = forward_elim(A,b,n)
    if augm_U["error"] != -1:
        X = backward_move(augm_U, n, m)
        augm_matrix = {"A":X["A"],
                       "b":X["b"],
                       "X":X["x"]}
        system_print(augm_matrix, settings = 0)
        return X
    else:
        augm_matrix = {"A":X["A"],
                       "b":X["b"]}
        system_print(augm_matrix, settings = 0)
        return U

# _Factorization_ procedure

In [15]:
def LU_decomp(A,b,n):
    
    LU = np.identity(n)
    error = 1
    increm_error = -1
    increm = 0

    LU[0,0:] = A[0,0:]
    if np.round_(LU[0,0],12) != 0:
        LU[1:,0] = A[1:,0]/LU[0,0]
    else: 
        print("There is a division by zero. The pivot u"+str(increm+1)+str(increm+1)+"=0.\n\
The system should be rearranged, as it could have unique, infinitely many solutions or none.")
        error = -1
        increm_error = increm
        
    for increm in range(1,n):
        LU[increm,increm:n]=A[increm,increm:n]-np.dot(LU[increm,0:increm],LU[0:increm,increm:n])
        LU_matrix = {"A":LU}
        system_print(LU_matrix, settings = 1)
        if np.round_(LU[increm,increm],12) != 0:
            LU[increm+1:n,increm] = (A[increm+1:n,increm]-np.dot(LU[increm+1:n,0:increm],LU[0:increm,increm]))/LU[increm,increm]
        else: 
            print("There is a division by zero. The pivot u"+str(increm+1)+str(increm+1)+"=0.\n\
The system should be rearranged, as it could have unique, infinitely many solutions or none.")
            error = -1
            increm_error = increm
            break
    
    return LU, error, increm_error

In [16]:
def LU_decomposition(augm_matrix):

    A = augm_matrix["A"].copy()
    b = augm_matrix["b"].copy()
    n = augm_matrix["n"]
    m = augm_matrix["m"]
    sign = 1
    
    LU, error, increm_error = LU_decomp(A,b,n)
    if error == -1:
        print("\nThe second try.")
        A, b = sort_by_row(A, b, increm_error, increm_error+1)
        LU, error, increm_error_2 = LU_decomp(A,b,n)
        sign *= -1
        
    
    LU = {"LU":LU,
          "b":b,
          "n":n,
          "m":m,
          "sign":sign,
          "increm_error":increm_error}

    return LU

# Obtaining the inverse of L and U

In [17]:
def solve_system(augm_matrix):
    
    '''
    Loading b = I identity matrix and LU matrix
    '''
    b = augm_matrix["b"]
    LU = augm_matrix["A"]
    
    '''
    Defining the shape of L and U matrixes
    '''
    n = LU.shape[0]
    m = LU.shape[1]

    '''
    Initializing L and U matrixes and its inverse matrixes
    '''
    L = np.identity(n)
    U = np.identity(n)
    
    U_inv = np.zeros((n,m))
    L_inv_t = np.zeros((n,m))
    
    '''
    Exctracting L and U matrixes
    '''
    for i in range(1,n):
        L[i:n,i-1] = LU[i:n,i-1]

    for i in range(0,n):
        U[i,i:n] = LU[i,i:n]
    
    L_t = np.transpose(L)

    '''
    Calculating inverse matrixes
    '''
    
    L_inv_t[m-1,] = b[n-1,]/L_t[n-1,m-1]
    for j in reversed(range(n-1)):
        L_inv_t[j,] = (b[j,]-np.dot(L_t[j,j:],L_inv_t[j:,]))/L_t[j,j]
    
    L_inv = np.transpose(L_inv_t)
    
    U_inv[m-1,] = b[n-1,]/U[n-1,m-1]
    for j in reversed(range(n-1)):
        U_inv[j,] = (b[j,]-np.dot(U[j,j:],U_inv[j:,]))/U[j,j]

    return L_inv, U_inv

# Obtaining the inverse of A

In [18]:
def inverse(augm_matrix):
    
    A_inv = np.zeros((augm_matrix["n"],augm_matrix["n"]))
    I = np.identity(augm_matrix["n"])
    LU_matrix = {"A":augm_matrix["LU"],
                 "b":I,
                 "sign":augm_matrix["sign"]}
    
    detU = determinant(LU_matrix)
    print("\nThe determinant is "+str(detU))

    if round(detU) != 0:
        L_inv, U_inv = solve_system(LU_matrix)
        A_inv = np.dot(U_inv, L_inv)
        if augm_matrix["increm_error"] >= 0:
            A_inv, _ = sort_by_col(A_inv, augm_matrix["increm_error"], augm_matrix["increm_error"]+1)
        system_print({"A":A_inv}, settings = 1)
    else:
        print("\nThe determinant of the matrix is zero, therefore the inverse matrix doesn't exist")
    
    return A_inv

# Conditionality of A

In [19]:
def condA(A, A_inv, norm = 0):
    '''
    type = 0 - The first norm
    type = 1 - the cubic norm
    '''
    cond = 0
    
    if np.sum(A_inv == 0) != A_inv.shape[0]**2:
        max_A = np.sum(np.abs(A), axis=norm).max()
        max_A_inv = np.sum(np.abs(A_inv), axis=norm).max() 
        cond = max_A * max_A_inv
    else:
        print("\nThere isn't the inverse matrix.")
        
    return cond

# The matrix method for solving the system of equations

In [20]:
def matrix_method(A_inv, b):
    
    x=[]
    if np.sum(A_inv == 0) != A_inv.shape[0]**2:
        x = np.dot(A_inv, b)
        print(np.round_(pd.DataFrame(x),12))
    else:
        print("\nThe inverse matrix doesn't exist.")
    
    return x

# The procedure that computes everything for this lab

In [21]:
def lab(augm_matrix={}):
    
    if augm_matrix=={}:
        '''
        Initializing the augmented matrix A
        '''
        augm_matrix = initializing_system()
    '''
    Solving the system by Gaussian elimination
    '''
    print("\nGaussian elimination")
    X = Gauss(augm_matrix)
    error(augm_matrix,X)
    '''
    LU decomposition
    '''
    print("\nLU decomposition")
    LU = LU_decomposition(augm_matrix)
    '''
    Obtaining the inverse matrix
    '''
    print("\nThe inverse matrix")
    Inverse_matrix = inverse(LU)
    '''
    Solving the system
    '''
    print("\nSolving the system by matrix method")
    matrix_method(Inverse_matrix, augm_matrix["b"])
    '''
    With disturbance
    '''
    print("\nWith disturbance")  
    b_copy = (augm_matrix["b"]).copy()
    b_copy *= 1.01
    '''
    Solving for LU decomposition
    '''
    print("\nSolving the system by LU decomposition")
    matrix_method(Inverse_matrix, b_copy)
    '''
    Solving for Gaussian elimination
    '''
    print("\nSolving the system by Gaussian elimination")
    augm_matrix_dist = augm_matrix.copy()
    augm_matrix_dist["b"] = b_copy
    Gauss(augm_matrix_dist)   
    '''
    Conditionality of the system
    '''
    print("\nThe first norm cond(A)="+str(condA(augm_matrix["A"], Inverse_matrix, norm = 0)))
    print("\nThe second norm cond(A)="+str(condA(augm_matrix["A"], Inverse_matrix, norm = 1)))
    
    print("The determinant of A by numpy module:\n"+str(np.linalg.det(augm_matrix["A"])))
    print("\nThe column matrix X by numpy module:\n"+str(pd.DataFrame(np.round_(np.linalg.solve(augm_matrix["A"], augm_matrix["b"]),12))))
    print("The inverse of A by numpy module:\n"+str(pd.DataFrame(np.round_(np.linalg.inv(a=augm_matrix["A"]),12))))
    return 

# Task 1

In [22]:
lab()


Attention! This function processes only a square matrix of the coefficients of the system of equations 
with the column matrix of the constant terms.

Please, enter the number of equations.
number_of_equations = 3

Please, enter the number of variables.
number_of_variables = 3

Initialized matrix A
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]

Please, enter elements of the matrix A by columns
a11= 8
a12= 2
a13= -3
a21= 2
a22= 10
a23= 5
a31= 1
a32= -1
a33= 16

The matrix A
[[  8.   2.  -3.]
 [  2.  10.   5.]
 [  1.  -1.  16.]]

Initialized matrix b
[[ 0.]
 [ 0.]
 [ 0.]]

Please, enter elements of the matrix b
b1= 10
b2= 12
b3= 0

The column matrix b
[[ 10.]
 [ 12.]
 [  0.]]

Gaussian elimination
Pivot a11 is not zero
[[  8.      2.     -3.   ]
 [  0.      9.5     5.75 ]
 [  0.     -1.25   16.375]]
Pivot a22 is not zero
[[  8.           2.          -3.        ]
 [  0.           9.5          5.75      ]
 [  0.           0.          17.13157895]]

The matrix A is nonsingular
The matrix A

# Task 2

In [24]:
lab()


Attention! This function processes only a square matrix of the coefficients of the system of equations 
with the column matrix of the constant terms.

Please, enter the number of equations.
number_of_equations = 3

Please, enter the number of variables.
number_of_variables = 3

Initialized matrix A
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]

Please, enter elements of the matrix A by columns
a11= 4
a12= -1
a13= 2
a21= -1
a22= 4
a23= 2
a31= 3
a32= 3
a33= 4

The matrix A
[[ 4. -1.  2.]
 [-1.  4.  2.]
 [ 3.  3.  4.]]

Initialized matrix b
[[ 0.]
 [ 0.]
 [ 0.]]

Please, enter elements of the matrix b
b1= 3
b2= 3
b3= 6

The column matrix b
[[ 3.]
 [ 3.]
 [ 6.]]

Gaussian elimination
Pivot a11 is not zero
[[ 4.   -1.    2.  ]
 [ 0.    3.75  2.5 ]
 [ 0.    3.75  2.5 ]]
Pivot a22 is not zero
[[ 4.   -1.    2.  ]
 [ 0.    3.75  2.5 ]
 [ 0.    0.    0.  ]]

The matrix A is nonsingular
There are infinitely many solutions. The last pivot is zero.
Please, enter random number on your choice.
x2=0

IndexError: index 3 is out of bounds for axis 0 with size 3

# The first matrix A

In [38]:
augm_A = initializing_system()


Attention! This function processes only a square matrix of the coefficients of the system of equations 
with the column matrix of the constant terms.

Please, enter the number of equations.
number_of_equations = 4

Please, enter the number of variables.
number_of_variables = 4

Initialized matrix A
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]

Please, enter elements of the matrix A by columns
a11= 2
a12= 2
a13= -1
a14= 1
a21= -3
a22= 0
a23= 3
a24= 0
a31= -1
a32= 3
a33= 3
a34= 2
a41= 1
a42= 0
a43= 0
a44= 4

The matrix A
[[ 2.  2. -1.  1.]
 [-3.  0.  3.  0.]
 [-1.  3.  3.  2.]
 [ 1.  0.  0.  4.]]

Initialized matrix b
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]

Please, enter elements of the matrix b
b1= 3
b2= -9
b3= -7
b4= 4

The column matrix b
[[ 3.]
 [-9.]
 [-7.]
 [ 4.]]


# The second matrix E

In [32]:
augm_E = initializing_system()


Attention! This function processes only a square matrix of the coefficients of the system of equations 
with the column matrix of the constant terms.

Please, enter the number of equations.
number_of_equations = 4

Please, enter the number of variables.
number_of_variables = 4

Initialized matrix A
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]

Please, enter elements of the matrix A by columns
a11= -5
a12= 0
a13= 7
a14= 0
a21= 4
a22= -24
a23= 0
a24= 1
a31= 3
a32= 12
a33= -7
a34= -23
a41= -2
a42= 42
a43= 37
a44= -21

The matrix A
[[ -5.   0.   7.   0.]
 [  4. -24.   0.   1.]
 [  3.  12.  -7. -23.]
 [ -2.  42.  37. -21.]]

Initialized matrix b
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]

Please, enter elements of the matrix b
b1= 109
b2= 168
b3= -193
b4= -95

The column matrix b
[[ 109.]
 [ 168.]
 [-193.]
 [ -95.]]


# The third matrix L

In [33]:
augm_L = initializing_system()


Attention! This function processes only a square matrix of the coefficients of the system of equations 
with the column matrix of the constant terms.

Please, enter the number of equations.
number_of_equations = 4

Please, enter the number of variables.
number_of_variables = 4

Initialized matrix A
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]

Please, enter elements of the matrix A by columns
a11= 3
a12= -2
a13= -7
a14= -1
a21= 7
a22= -10
a23= -5
a24= 1
a31= 4
a32= 0
a33= -15
a34= -9
a41= -8
a42= 8
a43= 13
a44= 4

The matrix A
[[  3.  -2.  -7.  -1.]
 [  7. -10.  -5.   1.]
 [  4.   0. -15.  -9.]
 [ -8.   8.  13.   4.]]

Initialized matrix b
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]

Please, enter elements of the matrix b
b1= 2
b2= 28
b3= -21
b4= -11

The column matrix b
[[  2.]
 [ 28.]
 [-21.]
 [-11.]]


In [34]:
lab(augm_A)


Gaussian elimination
Pivot a11 is not zero
[[ 2.   2.  -1.   1. ]
 [ 0.   3.   1.5  1.5]
 [ 0.   4.   2.5  2.5]
 [ 0.  -1.   0.5  3.5]]
Pivot a22 is not zero
[[ 2.   2.  -1.   1. ]
 [ 0.   3.   1.5  1.5]
 [ 0.   0.   0.5  0.5]
 [ 0.   0.   1.   4. ]]
Pivot a33 is not zero
[[ 2.   2.  -1.   1. ]
 [ 0.   3.   1.5  1.5]
 [ 0.   0.   0.5  0.5]
 [ 0.   0.   0.   3. ]]

The matrix A is nonsingular
The matrix A
     0    1    2    3
0  2.0  2.0 -1.0  1.0
1  0.0  3.0  1.5  1.5
2  0.0  0.0  0.5  0.5
3  0.0  0.0  0.0  3.0
The column matrix b
     0
0  3.0
1 -4.5
2  0.5
3  0.0
The column matrix of unknown variables x
     0
0  4.0
1 -2.0
2  1.0
3  0.0

LU decomposition
The matrix A
     0    1    2    3
0  2.0  2.0 -1.0  1.0
1 -1.5  3.0  1.5  1.5
2 -0.5  0.0  1.0  0.0
3  0.5  0.0  0.0  1.0
The matrix A
     0         1    2    3
0  2.0  2.000000 -1.0  1.0
1 -1.5  3.000000  1.5  1.5
2 -0.5  1.333333  0.5  0.5
3  0.5 -0.333333  0.0  1.0
The matrix A
     0         1    2    3
0  2.0  2.000000 -1.0

In [35]:
lab(augm_E)


Gaussian elimination
Pivot a11 is not zero
[[ -5.    0.    7.    0. ]
 [  0.  -24.    5.6   1. ]
 [  0.   12.   -2.8 -23. ]
 [  0.   42.   34.2 -21. ]]
Pivot a22 is not zero
[[ -5.     0.     7.     0.  ]
 [  0.   -24.     5.6    1.  ]
 [  0.     0.     0.   -22.5 ]
 [  0.     0.    44.   -19.25]]
Pivot a33 is not zero
[[ -5.     0.     7.     0.  ]
 [  0.   -24.     5.6    1.  ]
 [  0.     0.    44.   -19.25]
 [  0.     0.     0.   -22.5 ]]

The matrix A is nonsingular
The matrix A
     0     1     2      3
0 -5.0   0.0   7.0   0.00
1  0.0 -24.0   5.6   1.00
2  0.0   0.0  44.0 -19.25
3  0.0   0.0   0.0 -22.50
The column matrix b
       0
0  109.0
1  255.2
2  308.0
3   -0.0
The column matrix of unknown variables x
      0
0 -12.0
1  -9.0
2   7.0
3   0.0

LU decomposition
The matrix A
     0     1    2    3
0 -5.0   0.0  7.0  0.0
1 -0.8 -24.0  5.6  1.0
2 -0.6   0.0  1.0  0.0
3  0.4   0.0  0.0  1.0
The matrix A
     0      1    2     3
0 -5.0   0.00  7.0   0.0
1 -0.8 -24.00  5.6   1.0
2

In [36]:
lab(augm_L)


Gaussian elimination
Pivot a11 is not zero
[[  3.          -2.          -7.          -1.        ]
 [  0.          -5.33333333  11.33333333   3.33333333]
 [  0.           2.66666667  -5.66666667  -7.66666667]
 [  0.           2.66666667  -5.66666667   1.33333333]]
Pivot a22 is not zero
[[  3.          -2.          -7.          -1.        ]
 [  0.          -5.33333333  11.33333333   3.33333333]
 [  0.           0.           0.          -6.        ]
 [  0.           0.           0.           3.        ]]
Pivot a33 is not zero
[[  3.          -2.          -1.          -7.        ]
 [  0.          -5.33333333   3.33333333  11.33333333]
 [  0.           0.          -6.           0.        ]
 [  0.           0.           0.           0.        ]]
The system is singular and has infinitely many solutions. The last pivot is zero.
Please, enter random number on your choice.
x3=0
The matrix A
     0         1         2          3
0  3.0 -2.000000 -1.000000  -7.000000
1  0.0 -5.333333  3.333333  1

LinAlgError: Singular matrix

# Results discussion

1. Two methods were used for solving the system of equations. As a result, we've got correct column matrix X, the determinant of A and the inverse of A in comparison with the solution by numpy module.
2. If there exist a zero pivot, then the LU decomposition doesn't work (we used LUP decomposition instead of LU decomp.), but the Gaussian elimination may still work. Therefore, in terms of a possibility to solve the system, Gaussian elimination is preferred, but in terms of a speed, the LU decomposition is better.
3. Below, we can see timing of two methods.

|Method|Time|
|-------------------|--------|
|Gaussian elimination|294.017 ms|
|LU decomposition|192.011 ms|